# MRI VISUALIZATION from OASIS-3
### Jack Le - Polygence

## Data Use Agreements
The data used for this project were provided in part by OASIS and ADNI.

OASIS-3: Principal Investigators: T. Benzinger, D. Marcus, J. Morris; NIH P50 AG00561, P30 NS09857781, P01 AG026276, P01 AG003991, R01 AG043434, UL1 TR000448, R01 EB009352. AV-45 doses were provided by Avid Radiopharmaceuticals, a wholly owned subsidiary of Eli Lilly.

Data collection for this project was done through the Alzheimer's Disease Neuroimaging Initiative (ADNI) (National Institutes of Health Grant U01 AG024904) and DOD ADNI (Department of Defense award number W81XWH-12-2-0012). ADNI is funded by the National Institute on Aging, the National Institute of Biomedical Imaging and Bioengineering, and through generous contributions from the following: AbbVie, Alzheimer’s Association; Alzheimer’s Drug Discovery Foundation; Araclon Biotech; BioClinica, Inc.; Biogen; Bristol-Myers Squibb Company; CereSpir, Inc.; Cogstate; Eisai Inc.; Elan Pharmaceuticals, Inc.; Eli Lilly and Company; EuroImmun; F. Hoffmann-La Roche Ltd and its affiliated company Genentech, Inc.; Fujirebio; GE Healthcare; IXICO Ltd.; Janssen Alzheimer Immunotherapy Research & Development, LLC.; Johnson & Johnson Pharmaceutical Research & Development LLC.; Lumosity; Lundbeck; Merck & Co., Inc.; Meso Scale Diagnostics, LLC.; NeuroRx Research; Neurotrack Technologies; Novartis Pharmaceuticals Corporation; Pfizer Inc.; Piramal Imaging; Servier; Takeda Pharmaceutical Company; and Transition Therapeutics. The Canadian Institutes of Health Research is providing funds to support ADNI clinical sites in Canada. Private sector contributions are facilitated by the Foundation for the National Institutes of Health (www.fnih.org). The grantee organization is the Northern California Institute for Research and Education, and the study is coordinated by the Alzheimer’s Therapeutic Research Institute at the University of Southern California. ADNI data are disseminated by the Laboratory for Neuro Imaging at the University of Southern California.

In [1]:
import nibabel.freesurfer.mghformat as mgh
import ipywidgets as widgets
from scipy import ndimage
import os, random
import matplotlib.gridspec as gridspec
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython import display as dspl

## MRI Visualizations

In [2]:
from skimage.transform import resize, rescale

In [3]:
# chooses a random MRI based on the chosen patient type
def choose_image(p_type):
    # Chooses a random image from the chosen group
    print(p_type)
    
    label_dir = "/home/jack/Code/GitHub/Polygence/Data/OASIS/mri_data/dementia" if p_type == 'Demented' else "/home/jack/Code/GitHub/Polygence/Data/OASIS/mri_data/control"
    mr_id = random.choice(os.listdir(label_dir))
    scans = os.path.join(label_dir, mr_id)
    img_file = [file for file in os.listdir(scans) if "transformed" in file]
    scan_dir = os.path.join(scans, img_file[0])

    # Loads image
    image_obj = mgh.load(scan_dir)
    image_data = image_obj.get_fdata()
    
    # Data preprocessing
    def normalize(volume):
        """ Normalize the volume, scaling it to [0, 1] instead of [0, 255] """
        min = 0.0
        max = 255.0
        volume[volume < min] = min
        volume[volume > max] = max
        volume = (volume - min) / (max - min)
        volume = volume.astype("float32")
        return volume

    def scale(volume):
        """ Reduce the volume's dimensions """
        return rescale(volume, .5)
    
    image_data = normalize(image_data)
    image_data = scale(image_data)

    return scan_dir, image_data

In [4]:
# graphs the MRI image
def show_graph(img, pat_type, height, width, depth, color):
    # Plots brain scans in all three dimensions
    fig = plt.figure(figsize=(15,7))
    gs = gridspec.GridSpec(nrows=2, ncols=3)

    ax0 = fig.add_subplot(gs[0:2,0])
    ax1 = fig.add_subplot(gs[0:2,1])
    ax2 = fig.add_subplot(gs[0:2,2])

    ax0.set_title("height")
    ax1.set_title("width")
    ax2.set_title("depth")

    colormap = 'gray' if color == 'Gray' else 'viridis'
    ax0.imshow(img[0, height, :, :], cmap=colormap)
    ax1.imshow(img[0, :, width, :], cmap=colormap)
    ax2.imshow(img[0, :, :, depth], cmap=colormap)

## Interactive MRI Display
### This may not display correctly on GitHub; to use it, you have to download the file and run it on your own computer

In [5]:
def disp(d_scan_dir, d_pat_type, d_image_data):
    print(f'Path of current MRI: {d_scan_dir}')
    _, d_high, d_wid, d_dep  = d_image_data.shape
    print(d_image_data.shape)
    co = widgets.ToggleButtons(options=['Gray', 'Color'], description='Color Map')
    hi = widgets.IntSlider(value=d_high/2, max=d_high-1, continous_updates=False, description='View 1')
    wi = widgets.IntSlider(value=d_wid/2, max=d_wid-1, continous_updates=False, description='View 2')
    de = widgets.IntSlider(value=d_dep/2, max=d_dep-1, continous_updates=False, description='View 3')
    ui = widgets.HBox([hi, wi, de, co])
    
    display_output = widgets.interactive_output(show_graph, {'img':widgets.fixed(d_image_data),
                                                             'pat_type':widgets.fixed(d_pat_type), 
                                                             'height':hi,
                                                             'width':wi,
                                                             'depth':de,
                                                             'color':co})
    dspl.display(ui, display_output)

In [6]:
from monai.transforms import BoundingRect
from monai.transforms import SpatialCrop
from monai.transforms import AddChannel

def show_display(pat_type):
    scan_dir, image_data = choose_image(pat_type)
#     image_data = image_data[30:220, :, :]
    height, width, depth = image_data.shape
#     image_data = image_data[int(30/256 * height):int(220/256 * height), 
#                             int(30/256 * width):int(220/256 * width), 
#                             int(0/256 * depth):int(220/256 * depth)]


    channel = AddChannel()
    image_data = channel(image_data)
    
#     cropper = CropForeground()
#     image_data = cropper(image_data)
    cropper = SpatialCrop(roi_start=(15, 15, 0), roi_end=(110,110,110))
    image_data = cropper(image_data)

    disp(scan_dir, pat_type, image_data)

### Cognitively Normal Patient Display

In [7]:
show_display("Control")

Control
Path of current MRI: /home/jack/Code/GitHub/Polygence/Data/OASIS/mri_data/control/OAS30327_MR_d0281/OAS30327_Freesurfer53_d0281_brain_transformed.mgz
(1, 95, 95, 110)


Output()

### Demented Patient Display

In [8]:
show_display("Demented")

Demented
Path of current MRI: /home/jack/Code/GitHub/Polygence/Data/OASIS/mri_data/dementia/OAS30732_MR_d0074/OAS30732_Freesurfer53_d0074_brain_transformed.mgz
(1, 95, 95, 110)


Output()

In [9]:
print((10,9,12) > (10,10,10))

False
